In [132]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '../../'))

In [133]:
import importlib
importlib.reload(sys.modules['scripts.data_loading'])

<module 'scripts.data_loading' from '/home/kaleb/projects/kaim/KAIM-Week1/notebooks/eda/../../scripts/data_loading.py'>

In [134]:
from scripts.data_loading import load_dataset

In [137]:
data = load_dataset('../../data/raw_analyst_ratings.csv')


In [138]:
data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
